In [1]:
import sys

sys.path.append("../")

In [4]:
from stock_data import StocksList, StockData, StocksDataTask
from stock_data.samplers import SimpleStockDataSampler
from targets import ClosePriceTargetFunction
from features import IndicatorsFeaturesList, FeatureFactory
from models import GBMRegressionModel

from sklearn.metrics import mean_absolute_percentage_error

Firstly, we need to read data and add features to it. `StockData` class is responsible for reading the stock data from file and `FeatureFactory` creates features.

In [7]:
data = StockData("AAPL", data_path="../data/AAPL.csv")
data = FeatureFactory.preprocess(data, IndicatorsFeaturesList())

Now, we need to define our targets. `Target` is a class which contains target data for certain `StockData`. It is created when we call `get` method of `TargetFunction` class.  
In the future we will need `StocksDataTask` for keeping `StockData` and `Target` together. Moreover, as far as we can train our model for multiple features, we need to create `StocksList` and `TargetsList`.

In [8]:
target_function = ClosePriceTargetFunction(bars_count=10)

stocks_list = StocksList([data])
targets_list = target_function.from_stocks_list(stocks_list)

task_data = StocksDataTask(stocks_list, targets_list)

In this moment we can split our data into train and test parts. `SimpleStockDataSampler` here is used to make data ready for passing it to the `GBMRegressionModel`.

In [10]:
train_data, test_data = task_data.split(left_side_ratio=0.2)

sampler = SimpleStockDataSampler()
X_train, y_train = sampler.sample(train_data)
X_test, y_test = sampler.sample(test_data)

Finally, we can train our model and measure MAPE on the test sample.

In [11]:
model = GBMRegressionModel()
model.fit(X_train, y_train, verbose=False)

predictions = model.predict(X_test)

metric = mean_absolute_percentage_error(y_test, predictions)
print(f"MAPE: {metric}")

MAPE: 0.10311060786645385
